In [1]:
import os
from pathlib import Path

import h5py
import numpy as np
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [140]:
select_axis = 2
select_res = 10
data = Path(os.getenv('DATA'))/'nako'
h5_path = Path(data/f'interim/t1_pp_{select_res}_cropped.h5')

In [141]:
info_df = pd.read_csv(data/'interim/nako_age_labels.csv', dtype={'key': 'string'})
info_df = info_df.set_index('key')

In [142]:
transpose = [[0, 1, 2], [1, 0, 2], [2, 0, 1]]
with h5py.File(h5_path, 'r') as hf:
    keys = list(hf['image'])
    sample_img = hf['image'][keys[0]][:]
    sample_img = np.transpose(sample_img, axes=transpose[select_axis])
    shape = np.array(sample_img.shape)

In [143]:
print(f'keys {len(keys)} - selected axis {select_axis}\nshape (transposed) {shape}')

keys 10692 - selected axis 2
shape (transposed) [155 155 185]


In [144]:
# create dataframe
indices = np.arange(shape[0]*len(keys))
subject_keys = np.array(keys)[indices//shape[0]]
df = pd.DataFrame({'key': subject_keys,
                   'slice': indices%shape[0],
                   'age': info_df.loc[subject_keys]['age'].to_numpy().astype(np.uint8),
                   'sex': info_df.loc[subject_keys]['sex'].to_numpy().astype(np.uint8)},
                   index=indices)
df['key'] = df['key'].astype('string')
df

,key,slice,age,sex
0,100000,0,52,1
1,100000,1,52,1
2,100000,2,52,1
3,100000,3,52,1
4,100000,4,52,1
...,...,...,...,...
1657255,111680,150,65,2
1657256,111680,151,65,2
1657257,111680,152,65,2
1657258,111680,153,65,2


In [145]:
# save to feather
df.to_feather(data/f'interim/t1_pp_{select_res}_slices_ax{select_axis}.feather')